In [11]:
import pandas as pd
import numpy as np

import time
import sys
from tqdm.notebook import tqdm
import gc

pd.set_option('display.max_colwidth', -1)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt


In [2]:
path_data = "/Users/yohannlefaou/Documents/data/posos/"

In [3]:
train = pd.read_csv(path_data +"input_train.csv", sep=",")
print(train.shape)

(8028, 2)


In [4]:
labels = pd.read_csv(path_data + "output_train.csv", sep=",")

In [10]:
train.head(50)

,ID,question
0,0,"bonjour, je m suis trompé de forum pour ma question alors je la repose ici. je pris pour la première fois hier du paroxétine et ce matin c'est une catastrophe. picotement dasn tous le corps annonciateur de sueur froide très très massive et de vomissement. j'en suis à deux crises depuis 5 heure du mat. la cela semble passer mes mes mains reste moites et chaude estce normal pour la première fois merci a tous"
1,1,est ce que le motilium me soulagera contre les nausées?
2,2,"mon médecin m'a prescrit adenyl. au 2ème cachet des maux de tête terribles et au 3ème palpitations, sueurs froides, chaleur intense dans la tête, tremblements, fourmillements dans la lèvre supérieure, difficultés à respirer.. dès l'arrêt du médicament tous les symptômes ont disparu. cela est-il déjà arrivé à quelqu'un??"
3,3,Est-ce qu'il existe une forme adaptée aux enfant de 5ans du Micropakine ?
4,4,mon medecin me soigne pour une rhino pharingite et m'a prescrit du amoxicilline comme anti biotique. Est-ce vraiment pour cette indication?
5,5,je viens d'apprendre que je suis enceinte..savez-vous si je peux poursuivre le rubozinc ?
6,6,atrax n'est-il pas dangereux au long terme ?
7,7,je suis sous mercilon. J'ai des nausées et des saignements ?
8,8,"L'atenolol, c'est quoi ?"
9,9,"je prend trinordiol et à la fin de ma première plaquette j'ai eu ma première fois ac mn copain. la pilule est donc bien efficace dès le debut ? j'ai des douleur dans la poitrine, j'ai l'impression qu'ils ont un peu grossi ossi de plus, j'ai mal o bas du ventr,e dois je minkiété ou c simplemen, la periode dadaptation ??"


In [9]:
labels.head(50)

,ID,intention
0,0,28
1,1,31
2,2,28
3,3,44
4,4,31
5,5,44
6,6,48
7,7,28
8,8,22
9,9,23


In [7]:
labels["intention"].value_counts()

28    1796
31    565 
44    560 
22    461 
48    387 
34    382 
32    321 
14    317 
42    315 
23    208 
0     204 
11    195 
21    180 
33    144 
8     134 
37    126 
38    120 
13    119 
27    104 
5     102 
43    91  
12    89  
47    82  
29    77  
4     76  
26    72  
10    68  
24    64  
9     54  
30    52  
35    50  
45    50  
41    45  
46    45  
39    40  
50    33  
25    30  
6     26  
36    25  
15    23  
1     23  
18    23  
2     22  
49    22  
7     20  
19    20  
20    19  
16    17  
17    15  
3     8   
40    7   
Name: intention, dtype: int64

In [14]:
label = 27
train.loc[labels["intention"].values == label, :].head(10)

,ID,question
90,90,"bonjour, je me suis réveillée dimanche matin avec de l'urticaire sur les jambes et un peu sur les bras.. je suis allée voir mon médecin hier, il en a déduit que c'était une allergie au stresam (j'ai arrêté mon traitement dès dimanche matin) et m'a prescrit poralamine. j'en ai pris 2 hier soir mais en me levant ce matin, j'ai vu que j'avais à présent de l'urticaire sur le décolleté et sur les bras et les mains. c'est bien moins important que sur les jambes mais quand même! je m'inquiète : est-ce normal que l'urticaire continue de ""monter"" malgré le poralamine? dois-je craindre un oedème de quincke? merci"
175,175,pdt combien de temps les effets secondaires.à l'arrêt du deroxat?
297,297,"si une mauvaise réaction ou si des complications suivent un vaccin contre l'hépatite b, combien de temps après l'injection cela arive t-il, du moins à partir de quand peut-on se dire qu'il n'y a pas de mauvaise réaction ou complication?"
318,318,je prends du bromazepam. combien de temps dure l'effet rebond décrit dans la notice ?
330,330,combien de temps dure les effets indésirables du vaccin ror ?
380,380,"je me susi fait poser un stérilet en cuivre flexi t voici 5 semaines. a mes premieres règles depuis la pose , qui ont été extremement abondantes, j'ai fait plusieurs malaises et baisses de tension. cela se calme t il avec le temps?"
395,395,au bout de combien de semaines les effets secondaires du seropram disparaissent ?
402,402,"xanax, ixel ,stilnox,les effets des 1er jours partent qd?"
413,413,"je me suis faite poser le stérilet mirena il y a 6 jours, et j'ai des petites pertes de sang ; est ce que ça devrait s'estomper tout seul et au bout de combien de temps ?"
429,429,je prend la pilule triella depuis 2semaines et j'ai l'impression que ma poitrine a déjà grossi. elle est plus galbé et dure contrairement à avant !! cela va t'il continuer ?? va t'elle encore grossir ou s’arrêter dans ls mois à venir ??


In [ ]:
class_description = {"28" : "effet indesirable, effet secondaire",
                     "31" : "interrogation sur adéquation d un medicament a une pathologie, phrase courte en general",
                     "44" : "contre indication des medicaments femme enceinte grossesse enfant bebe",
                     "22" : "demande sur qu est ce qu un medicament, tel medicament c est quoi etc., phrases courtes souvent",
                     "48" : "question sur les risques de médicaments, pilule",
                     "34": "questions sur les interractions de medicaments",
                     "26": "question sur la duree pendant laquelle un medicament agit et est efficace",
                     "27" : "combien de temps durent les effets indesirables d un medicament"
                    }



In [ ]:
typos = {"atrax" : "atarax",
        }

In [ ]:
'''
2 choses : voir le tokenizer
revoir les stop words
Elliot lui il n'enleve pas les stop words en general
'''